In [1]:
#pip install face_alignment

Wczytanie niezbędnych bibliotek

In [3]:
import cv2
import pandas as pd
import numpy as np


Pobranie zdjęcia z kamery, pobiera tylko twarz 

In [1]:
import cv2

# Inicjalizacja klasyfikatora do wykrywania twarzy
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Inicjalizacja kamery
cap = cv2.VideoCapture(0)
twarz = None  # Zmienna do przechowywania wyciętej twarzy

while True:
    # Pobranie obrazu z kamery
    ret, image = cap.read()
    
    if not ret:
        print("Nie udało się pobrać obrazu z kamery.")
        continue
    
    # Konwersja obrazu na odcienie szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Wykrywanie twarzy
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Jeżeli wykryto twarz, wytnij ją i zakończ pętlę
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            twarz = image[y:y+h, x:x+w]  # Wycinanie twarzy z obrazu
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        break
    
    # Wyświetlanie obrazu na żywo (opcjonalne)
    cv2.imshow('Zdjecie_z_kamery', image)
    
    # Naciśnij 'q', aby wyjść z pętli
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


# Zmienna twarz zawiera wyciętą twarz
if twarz is not None:
    cv2.imshow('Wycięta Twarz', twarz)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Nie wykryto frontalnej twarzy.")



In [2]:
twarz

array([[[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [207, 203, 192],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255,   0,   0]],

       ...,

       [[255,   0,   0],
        [255,   0,   0],
        [ 87,  94, 114],
        ...,
        [ 53,  49,  54],
        [ 53,  49,  54],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [ 85,  91, 113],
        ...,
        [ 52,  48,  53],
        [ 53,  49,  54],
        [255,   0,   0]],

       [[255,   0,   0],
        [255,   0,   0],
        [255,   0,   0],
        ...,
        [255,   0,   0],
        [255,   0,   0],
        [255,   0,   0]]

In [ ]:
#!pip install face-alignment
import face_alignment
import numpy as np
import pandas as pd

def detect_faces_and_landmarks_face_alignment(image: np.ndarray) -> pd.DataFrame:

    # Initialize the face_alignment detector
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu')

    # Detect landmarks
    predictions = fa.get_landmarks(image)
    
    if predictions is None:
        return pd.DataFrame(columns=['Face_Number', 'Landmark_Index', 'X', 'Y'])

    # Initialize a list to store landmarks for all detected faces
    landmarks_list = []

    # Iterate through predictions to populate the DataFrame
    for face_num, landmarks in enumerate(predictions):
        for landmark_num, landmark in enumerate(landmarks):
            x, y = landmark.astype(int)
            landmarks_list.append([face_num + 1, landmark_num + 1, x, y])

    # Create a DataFrame from the landmarks list
    landmarks_df = pd.DataFrame(landmarks_list, columns=['Face_Number', 'Landmark_Index', 'X', 'Y'])

    return landmarks_df


Zrobienie zmiennej z landmarkami

In [ ]:
landmarks_df = detect_faces_and_landmarks_face_alignment(twarz)

In [ ]:
landmarks_df

,Face_Number,Landmark_Index,X,Y
0,1,1,26,143
1,1,2,30,166
2,1,3,36,190
3,1,4,46,210
4,1,5,60,230
...,...,...,...,...
63,1,64,147,179
64,1,65,170,183
65,1,66,147,179
66,1,67,137,183


Narysuj landmarki na twarzy

In [ ]:

def draw_landmarks_on_image(image: np.ndarray, landmarks_df: pd.DataFrame, face_number: int) -> np.ndarray:
    # Filtruj DataFrame dla wybranej twarzy
    face_landmarks = landmarks_df[landmarks_df['Face_Number'] == face_number]

    # Iteruj przez wiersze filtrowanego DataFrame, rysując punkty na obrazie
    for index, row in face_landmarks.iterrows():
        # Pobierz współrzędne punktu
        x, y = int(row['X']), int(row['Y'])
        # Rysuj punkt na obrazie
        cv2.circle(image, (x, y), 2, (255, 0, 0), -1)

    return image

In [ ]:
twarz_zlandmarkami = draw_landmarks_on_image(twarz,landmarks_df,1)

In [14]:
cv2.imshow("Z landmarkami",twarz_zlandmarkami) #twarz_zlandmarkami - twarz_zlandmarkami

In [15]:
cv2.destroyAllWindows()


Kod Jednolity

In [1]:
import cv2
import face_alignment
import numpy as np
import pandas as pd

# Funkcja do detekcji twarzy i punktów charakterystycznych
def detect_faces_and_landmarks_face_alignment(image: np.ndarray) -> pd.DataFrame:
    # Initialize the face_alignment detector
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False, device='cpu')

    # Detect landmarks
    predictions = fa.get_landmarks(image)
    
    if predictions is None:
        return pd.DataFrame(columns=['Face_Number', 'Landmark_Index', 'X', 'Y'])

    # Initialize a list to store landmarks for all detected faces
    landmarks_list = []

    # Iterate through predictions to populate the DataFrame
    for face_num, landmarks in enumerate(predictions):
        for landmark_num, landmark in enumerate(landmarks):
            x, y = landmark.astype(int)
            landmarks_list.append([face_num + 1, landmark_num + 1, x, y])

    # Create a DataFrame from the landmarks list
    landmarks_df = pd.DataFrame(landmarks_list, columns=['Face_Number', 'Landmark_Index', 'X', 'Y'])

    return landmarks_df

# Funkcja do rysowania punktów charakterystycznych na obrazie
def draw_landmarks_on_image(image: np.ndarray, landmarks_df: pd.DataFrame, face_number: int) -> np.ndarray:
    # Filtruj DataFrame dla wybranej twarzy
    face_landmarks = landmarks_df[landmarks_df['Face_Number'] == face_number]

    # Iteruj przez wiersze filtrowanego DataFrame, rysując punkty na obrazie
    for index, row in face_landmarks.iterrows():
        # Pobierz współrzędne punktu
        x, y = int(row['X']), int(row['Y'])
        # Rysuj punkt na obrazie
        cv2.circle(image, (x, y), 2, (255, 0, 0), -1)

    return image

# Inicjalizacja klasyfikatora do wykrywania twarzy
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Inicjalizacja kamery
cap = cv2.VideoCapture(0)
twarz = None  # Zmienna do przechowywania wyciętej twarzy

while True:
    # Pobranie obrazu z kamery
    ret, image = cap.read()
    
    if not ret:
        print("Nie udało się pobrać obrazu z kamery.")
        continue
    
    # Konwersja obrazu na odcienie szarości
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Wykrywanie twarzy
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Jeżeli wykryto twarz, wytnij ją i zakończ pętlę
    if len(faces) > 0:
        for (x, y, w, h) in faces:
            twarz = image[y:y+h, x:x+w]  # Wycinanie twarzy z obrazu
            cv2.rectangle(image, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Zapisz obrazy do plików
        cv2.imwrite('zdjecie_z_kamery.jpg', image)
        cv2.imwrite('wycieta_twarz.jpg', twarz)
        
        break
    
    # Wyświetlanie obrazu na żywo (opcjonalne)
    cv2.imshow('Zdjecie_z_kamery', image)
    
    # Naciśnij 'q', aby wyjść z pętli
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Zakończenie pracy kamery
cap.release()
cv2.destroyAllWindows()

# Wyświetlenie zapisanych obrazów
image_from_file = cv2.imread('zdjecie_z_kamery.jpg')
face_from_file = cv2.imread('wycieta_twarz.jpg')

cv2.imshow('Zdjecie z kamery', image_from_file)
cv2.imshow('Twarz', face_from_file)

# Detekcja punktów charakterystycznych na wyciętej twarzy
if face_from_file is not None:
    landmarks_df = detect_faces_and_landmarks_face_alignment(face_from_file)
    print(landmarks_df)
    
    # Rysowanie punktów charakterystycznych na obrazie twarzy
    face_with_landmarks = draw_landmarks_on_image(face_from_file, landmarks_df, face_number=1)
    cv2.imshow('Twarz z Punktami Charakterystycznymi', face_with_landmarks)
else:
    print("Nie wykryto wyciętej twarzy.")

cv2.waitKey(0)
cv2.destroyAllWindows()


    Face_Number  Landmark_Index    X    Y
0             1               1   19   81
1             1               2   21  104
2             1               3   24  123
3             1               4   30  143
4             1               5   35  162
..          ...             ...  ...  ...
63            1              64  111  151
64            1              65  128  151
65            1              66  111  154
66            1              67  103  154
67            1              68   91  154

[68 rows x 4 columns]


KeyboardInterrupt: 